In [23]:
# Code cleaning helper, improves readability 
from pipe import *
# Tabulator libraries
import pandas as pd
import numpy as np
import panel as pn
import seaborn as sns
from IPython.display import HTML
pn.extension('tabulator')
import hvplot.pandas

# Web scraping libraries
from bs4 import BeautifulSoup, Tag
import requests
import lxml
import pandas as pd
import re

# Se pone pageSize = 72 para obtener todos los artoiculos dentro de la sección de Festival de Apple 
search_url = f"https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=RB2211-EPMTPE-CELULARES-FESTIVALAPPLE&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=0&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=72&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151"
# HTTP GET requests
page = requests.get(search_url)
# Revisa si se tuvo éxito obteniendo la URL 
if page.status_code == requests.codes.ok:
  print('Todo ok!')

Todo ok!


In [24]:
%%HTML
<style>.dataframe th,td:first-child{background: rgb(63,87,124);background: linear-gradient(180deg, rgba(63,87,124,1) 0%, rgba(101,124,161,1) 100%, rgba(0,212,255,1) 100%);;
padding: 10px;font-family: monospace;font-size: 110%;color: white;border:1px dashed white;text-align:left !important;
-moz-border-radius: 3x;-webkit-border-radius: 3px;}.dataframe thead{border:none; !important;}</style>

# Para que se vean bonitos los dataframes

In [25]:
bs = BeautifulSoup(page.text, 'lxml')
products = bs.findAll('div', class_='product_name')  # get all products
product_names = list( products | where(lambda x: type(x.p) == Tag) 
                               | select(lambda x: x.p) 
                               | select(lambda x: ''.join(x.find(text=True))) )# -> closes list
product_prices = bs.findAll('span', class_ = 'unique_price')
prices = list( product_prices  | select(lambda x: x.get_text() 
                               | strip('\r\n\t\t\t\t')) )# -> closes list
product_weekly_prices = bs.findAll('p', class_ = "catalog-entry-details") 
weekly_prices = list( product_weekly_prices  | select(lambda x: x.get_text() 
                               | strip('Desde\r\n\t\t\t\t\n\r\n\t\t\t\t\t  quincenal')) )# -> closes list
product_info = bs.findAll('div', {'class': 'product_info'})
hrefs = list( product_info     | select(lambda x: x.find("a").get("href")) )# -> closes list
skus = list(  hrefs            | select(lambda x: requests.get(x))
                               | select(lambda x: BeautifulSoup(x.text, "lxml"))
                               | select(lambda x: x.find('span', {'class': 'sku'}).get_text()) )# -> closes list

In [26]:
df_coppel = pd.DataFrame(
    {'Nombre del Artículo': product_names,
     'Precio': prices,
     '24 Pagos Quincenales' : weekly_prices,
     'SKU' : skus, 
     'Href' : hrefs
    })

In [27]:
# Convert Precio and 24 Pagos Quincenales colums from str to float 
df_coppel[df_coppel.columns[1:3]] = df_coppel[df_coppel.columns[1:3]].replace('[\$,]', '', regex=True).astype(float)

In [46]:
# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '11px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
  ]

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '11px')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]

# Set colormap equal to seaborns light green color palette
cm = sns.light_palette("#2e7dcc", as_cmap = True)

(df_coppel.style
  .background_gradient(cmap = cm, subset = ['Precio', '24 Pagos Quincenales'])
  .set_caption('Artículos de la promoción \"Festival de Apple\", vigencia del 21 al 27 de marzo del 2022.')
 .format({1: '$ {:.1f}', 1: '$ {:.1f}'}, na_rep = 'MISS', precision = 2)
  .set_table_styles(styles))

,Nombre del Artículo,Precio,24 Pagos Quincenales,SKU,Href
0,Telcel Apple iPhone 11 64 GB Black,13899.00,794.00,SKU:233299,https://www.coppel.com/telcel-apple-iphone-11-64-gb-black-pm-2332993
1,Telcel Apple iPhone 11 64 GB Purple,13899.00,794.00,SKU:232405,https://www.coppel.com/telcel-apple-iphone-11-64-gb-purple-pm-2324053
2,Apple Watch Series 7 41 mm Azul Medianoche,11299.00,646.00,SKU:252963,https://www.coppel.com/apple-watch-series-7-41-mm-azul-medianoche-pm-2529633
3,AirPods Apple Tercera Generación,4999.00,286.00,SKU:255474,https://www.coppel.com/airpods-apple-tercera-generacion-pm-2554743
4,Telcel Apple iPhone 12 64 GB Black,19199.00,1097.00,SKU:292971,https://www.coppel.com/telcel-apple-iphone-12-64-gb-black-pm-2929713
5,Apple Watch Series 7 41 mm Blanco Estelar,11299.00,646.00,SKU:252955,https://www.coppel.com/apple-watch-series-7-41-mm-blanco-estelar-pm-2529553
6,AirPods Pro con Estuche de Carga,6499.00,372.00,SKU:239256,https://www.coppel.com/airpods-pro-con-estuche-de-carga-pm-2392563
7,Apple Watch Series 7 41 mm Verde,11299.00,646.00,SKU:252939,https://www.coppel.com/apple-watch-series-7-41-mm-verde-pm-2529393
8,Apple Watch SE 40 mm Blanco Estrella,8499.00,486.00,SKU:248761,https://www.coppel.com/apple-watch-se-40-mm-blanco-estrella-pm-2487613
9,Apple Watch Series 7 41 mm Azul Abismo,11299.00,646.00,SKU:252947,https://www.coppel.com/apple-watch-series-7-41-mm-azul-abismo-pm-2529473
